In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Activation
from keras.metrics import AUC
from keras.callbacks import EarlyStopping, ModelCheckpoint
from matplotlib import pyplot

In [2]:
# Data Loading
print("Loading data...")
X_model = np.load('../../data/X_timeseries_expanded.npy')
Y_model = pd.read_csv('../../data/Y_model.csv').values

Loading data...


In [3]:
# Train Test Split
print("Splitting data...")
X_train, X_test, y_train, y_test = train_test_split(X_model, Y_model, test_size=0.2, random_state=42)

Splitting data...


In [4]:
# design network
# References: 
# - https://towardsdatascience.com/choosing-the-right-hyperparameters-for-a-simple-lstm-using-keras-f8e9ed76f046
# - https://stats.stackexchange.com/questions/242238/what-is-considered-a-normal-quantity-of-outliers
# - https://machinelearningmastery.com/multivariate-time-series-forecasting-lstms-keras/
hidden_nodes = int(2/3 * (X_train.shape[1] * X_train.shape[2]))
model = Sequential()
model.add(LSTM(hidden_nodes, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(1))
model.add(Activation('softmax'))
model.compile(
    loss='mae',
    optimizer='adam',
    metrics=[AUC()]
)

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-12-15 03:15:08.093829: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-12-15 03:15:08.098307: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [8]:
print("Training model...")
history = model.fit(
    X_train,
    y_train,
    epochs=50,
    batch_size=32,
    validation_data=(X_test, y_test), 
    verbose=1,
    shuffle=False,
    callbacks=[
        EarlyStopping(
            monitor='val_auc',
            mode='max',
            patience=3,
        ),
        ModelCheckpoint(
            'lstm.h5',
            save_best_only=True,
            monitor='val_auc',
            mode='max'
        )
    ]
)

Training model...
Epoch 1/50


2022-12-15 03:34:48.288089: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-12-15 03:34:49.038671: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-12-15 03:34:50.443031: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


  467/20000 [..............................] - ETA: 4:07:02 - loss: 0.9340 - auc: 0.5000

KeyboardInterrupt: 

In [ ]:
# plot history
pyplot.plot(history.history['auc'], label='train')
pyplot.plot(history.history['val_auc'], label='test')
pyplot.legend()
pyplot.show()

In [ ]:
# Save model with pickle
import pickle
with open("model.pkl", "wb") as f:
    pickle.dump(model, f)